In [1]:
import merge_pluto_finance_new as mpf

In [2]:
import pandas as pd
import numpy as np

In [3]:
boros = ["manhattan", "brooklyn", "bronx", "queens", "si"]
years = list(range(2003, 2017))

In [4]:
boros = ["".join(boro.lower().split()) for boro in boros]

In [5]:
pluto = mpf.read_in_pluto(boros)

In [6]:
finance = mpf.read_in_finance(boros, years)

Pulling Finance data for 2003_manhattan
Pulling Finance data for 2003_brooklyn
Pulling Finance data for 2003_bronx
Pulling Finance data for 2003_queens
Pulling Finance data for 2003_si
Pulling Finance data for 2004_manhattan
Pulling Finance data for 2004_brooklyn
Pulling Finance data for 2004_bronx
Pulling Finance data for 2004_queens
Pulling Finance data for 2004_si
Pulling Finance data for 2005_manhattan
Pulling Finance data for 2005_brooklyn
Pulling Finance data for 2005_bronx
Pulling Finance data for 2005_queens
Pulling Finance data for 2005_si
Pulling Finance data for 2006_manhattan
Pulling Finance data for 2006_brooklyn
Pulling Finance data for 2006_bronx
Pulling Finance data for 2006_queens
Pulling Finance data for 2006_si
Pulling Finance data for 2007_manhattan
Pulling Finance data for 2007_brooklyn
Pulling Finance data for 2007_bronx
Pulling Finance data for 2007_queens
Pulling Finance data for 2007_si
Pulling Finance data for 2008_manhattan
Pulling Finance data for 2008_brook

In [7]:
dtm = mpf.read_in_dtm(boros)

In [8]:
pluto.shape[0] - pluto.drop_duplicates().shape[0]

0

In [9]:
finance.shape[0] - finance.drop_duplicates().shape[0]

0

In [10]:
dtm.shape[0] - dtm.drop_duplicates().shape[0]

0

In [11]:
buildings = mpf.merge_pluto_finance(pluto, finance, dtm)

Updating lot numbers for condo units
Finance:(267610, 9) PLUTO:(812364, 41) DTM:(228319, 6)
Finance updated:(267610, 8)
Merging PLUTO with updated Dept. of Finance data


In [12]:
buildings.shape[0] - buildings.drop_duplicates().shape[0]

372

In [13]:
finance_updated = mpf.get_finance_condo_lot(pluto, finance, dtm)

In [14]:
finance_updated.shape[0] - finance_updated.drop_duplicates().shape[0]

626

In [15]:
dtm_cols_to_keep = ['unit_bbl', 'condo_boro', 'condo_numb']
pluto_cols_to_keep = ['bbl', 'block', 'borocode', 'condono']

finance_condos_only = pd.merge(finance, dtm[dtm_cols_to_keep],
    how='inner', left_on=['bbl'], right_on=['unit_bbl'])

# for condos: finance.bbl == finance_condos_only.unit_bbl
#             finance_condos_only.bbl_pluto == pluto.bbl
finance_condos_only = pd.merge(pluto[pluto_cols_to_keep],
    finance_condos_only, how='inner',
    left_on=['borocode', 'block', 'condono'],
    right_on=['condo_boro', 'block', 'condo_numb'],
    suffixes=['_pluto', '_finance'])

finance_condos_only['bbl_pluto'] = finance_condos_only['bbl_pluto'].astype(int)
finance_condos_only = finance_condos_only[['bbl_pluto', 'bbl_finance']]

In [16]:
finance_condos_only.shape[0] - finance_condos_only.drop_duplicates().shape[0]

358

In [17]:
finance_condos_only = finance_condos_only[
            ['bbl_pluto', 'bbl_finance']].drop_duplicates()

In [19]:
# get a list of bbls that are not condos (same in pluto and finance)
standard_bbls = list(set(finance.bbl).difference(
                    set(finance_condos_only.bbl_finance)))
# combine condo bbls that differ with standard bbls that are the same
bbl_mappings = finance_condos_only.append(pd.DataFrame.from_dict(
    {'bbl_pluto': standard_bbls, 'bbl_finance': standard_bbls}
))

In [20]:
finance_condo_updated = pd.merge(finance, bbl_mappings,
        how='left', left_on='bbl', right_on='bbl_finance')

In [23]:
finance_condo_updated.shape[0] - finance_condo_updated.drop_duplicates().shape[0]

626

In [22]:
finance_condo_updated = finance_condo_updated.drop(
        ['block','bbl', 'bbl_finance'], axis=1)

In [25]:
finance_condo_updated[finance_condo_updated.duplicated()]

,sale_price,sale_date,tax_class_at_time_of_sale,year_built,residential_units,commercial_units,total_units,bbl_pluto
291,350000,2014-10-30,1,1930,1,1,2,1000167502
943,455000,2008-08-25,1,1925,1,0,1,1000167506
993,45000,2016-02-10,4,1963,0,0,1,1000167506
1147,63000,2016-02-10,4,1963,0,0,1,1000167507
1273,45000,2016-02-10,4,1963,0,0,1,1000167508
1301,45000,2016-02-10,4,1963,0,0,1,1000167508
1357,37500,2016-02-10,4,1963,0,0,1,1000167508
1410,58900,2013-06-05,4,2007,0,2,2,1000167508
1495,0,2014-09-25,4,2007,0,2,2,1000167508
1523,0,2014-09-25,4,2007,0,2,2,1000167508


In [28]:
finance[finance.sale_price == 350000]

,sale_price,sale_date,tax_class_at_time_of_sale,year_built,residential_units,commercial_units,total_units,block,bbl
283242,350000,2005-10-28,1,0,1,0,1,8281,1000161173
1110265,350000,2014-10-30,1,1930,1,1,2,3821,1000162062
1110266,350000,2014-10-30,1,1930,1,1,2,3821,1000162086
1022490,350000,2013-10-24,1,1960,2,0,2,4402,1000162267
244253,350000,2004-12-20,1,1955,1,0,1,391,1000162745
694168,350000,2009-02-11,2,1962,0,0,0,909,1000162752
1141880,350000,2014-04-28,1,1920,1,0,1,16138,1000163033
1141886,350000,2014-06-30,1,1945,2,0,2,16123,1000163064
1231569,350000,2015-09-17,2,1972,0,0,0,8489,1000163081
659880,350000,2008-02-14,1,1925,1,0,1,3683,1000163096


In [31]:
buildings.yearbuilt

0         1900.0
1         1900.0
2         1900.0
3         1900.0
4         1900.0
5         1900.0
6         1900.0
7         1900.0
8         1900.0
9         1900.0
10        1900.0
11        1900.0
12        1900.0
13        1900.0
14        1970.0
15        1970.0
16        1970.0
17        1970.0
18        1970.0
19        1970.0
20        1970.0
21        1970.0
22        1970.0
23        1970.0
24        1970.0
25        1970.0
26        1970.0
27        1970.0
28        1970.0
29        1970.0
           ...  
198935    1935.0
198936    1935.0
198937    1935.0
198938    1935.0
198939    1962.0
198940    1962.0
198941    1962.0
198942    1962.0
198943    1962.0
198944    1959.0
198945    1959.0
198946    1959.0
198947    1959.0
198948    1959.0
198949    1935.0
198950    1935.0
198951    1935.0
198952    1935.0
198953    1935.0
198954    1987.0
198955    1987.0
198956    1987.0
198957    1987.0
198960    2003.0
198961    2003.0
198964    1941.0
198965    1941.0
198966    1941